In [1]:
"""
Solution of 2D Forward Problem of Linear Elasticity for Plane Stress Boundary Value Problem 
using Physics-Informed Neural Networks (PINN)
"""
import torch
import torch.nn as nn
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import plotly.graph_objects as go # for plotting the graph


In [2]:
torch.manual_seed(123456)
np.random.seed(123456)
E = 1                                       # Young's Modulus
nu = 0.3                                    # Poisson Ratio
G = E/( 2*(1+nu) )                          # Shear modulus

In [3]:
# Load the collocation point data (matlab file format .mat)
bp = scipy.io.loadmat('/kaggle/input/apl745-data/Lab 11 data/boundary_points.mat')
x_b = bp['x_bdry']
y_b = bp['y_bdry']
ip = scipy.io.loadmat('/kaggle/input/apl745-data/Lab 11 data/interior_points.mat')
x_col = ip['x']
y_col = ip['y']

x = np.concatenate((x_col, x_b), axis=0)
y = np.concatenate((y_col, y_b), axis=0)
r = x_col.shape[0]

u_b = torch.tensor(np.zeros((x_b.shape[0],1)), requires_grad=True, dtype=torch.float32)
v_b = torch.tensor(np.zeros((x_b.shape[0], 1)), requires_grad=True, dtype=torch.float32)

## Define data as PyTorch Tensor and send to device
x = torch.tensor(x, requires_grad=True, dtype=torch.float32)
y = torch.tensor(y, requires_grad=True, dtype=torch.float32)


In [4]:
class neural_network_model(nn.Module):
    def __init__(self):
        super(neural_network_model, self).__init__()
        
        self.layer1 = nn.Linear(2, 30)
        self.layer2 = nn.Linear(30, 30)
        self.layer3 = nn.Linear(30, 30)
        self.layer4 = nn.Linear(30, 30)
        self.layer5 = nn.Linear(30, 30)
        self.layer_out = nn.Linear(30, 2)
        self.tanh = nn.Tanh()

    def forward(self, x, y):
        s = self.tanh(self.layer1( torch.concat((x,y),dim=1)))
        s = self.tanh(self.layer2(s))
        s = self.tanh(self.layer3(s))
        s = self.tanh(self.layer4(s))
        s = self.tanh(self.layer5(s))
        s = self.layer_out(s)
        return s[:,0], s[:,1]

In [5]:
class PINN_Model:
    """A class used for the definition of Physics Informed Models for one dimensional bars."""

    def __init__(self, x, y, u_b, v_b, r):
        # Initialize required variables for the class
        self.x = x
        self.y = y
        self.u_b = u_b
        self.v_b = v_b
        self.r = r
    
    def loss( self, x,y, u_col_pred, v_col_pred, u_b_pred, v_b_pred):
            # Define the loss function for the model
            u_x = torch.autograd.grad(u_col_pred, self.x, grad_outputs=torch.ones_like(u_col_pred),create_graph=True, retain_graph=True )[0]
            u_y = torch.autograd.grad(u_col_pred, self.y, grad_outputs=torch.ones_like(u_col_pred),create_graph=True, retain_graph=True )[0]
            v_x = torch.autograd.grad(v_col_pred, self.x, grad_outputs=torch.ones_like(v_col_pred),create_graph=True, retain_graph=True )[0]
            v_y = torch.autograd.grad(v_col_pred, self.y, grad_outputs=torch.ones_like(v_col_pred),create_graph=True, retain_graph=True )[0]
            u_xx = torch.autograd.grad(u_x, self.x, grad_outputs=torch.ones_like(u_x),create_graph=True, retain_graph=True )[0]
            u_yy = torch.autograd.grad(u_y, self.y, grad_outputs=torch.ones_like(u_y),create_graph=True, retain_graph=True )[0]
            v_xx = torch.autograd.grad(v_x, self.x, grad_outputs=torch.ones_like(v_x),create_graph=True, retain_graph=True )[0]
            v_yy = torch.autograd.grad(v_y, self.y, grad_outputs=torch.ones_like(v_y),create_graph=True, retain_graph=True )[0]
            u_xy = torch.autograd.grad(u_x, self.y, grad_outputs=torch.ones_like(u_x),create_graph=True, retain_graph=True )[0]
            v_xy = torch.autograd.grad(v_x, self.y, grad_outputs=torch.ones_like(v_x),create_graph=True, retain_graph=True )[0]
            u_yx = torch.autograd.grad(u_y, self.x, grad_outputs=torch.ones_like(u_y),create_graph=True, retain_graph=True )[0]
            v_yx = torch.autograd.grad(v_y, self.x, grad_outputs=torch.ones_like(v_y),create_graph=True, retain_graph=True )[0]

            f_1 = torch.sin(2*torch.pi*x)*torch.sin(2*torch.pi*y)
            f_2 = torch.sin(torch.pi*x)*torch.sin(2*torch.pi*y) 
            df_loss_1 = G*( (u_xx+u_yy) + ((1+nu)/(1-nu))*( u_xx + v_yx ) ) + f_1
            df_loss_2 = G*( (v_xx+v_yy) + ((1+nu)/(1-nu))*( v_yy + u_xy ) ) + f_2
            diff_loss = torch.mean( (df_loss_1-0)**2 ) + torch.mean( (df_loss_2-0)**2 )
            bc_loss = torch.mean(torch.square(u_b_pred - self.u_b)) + torch.mean(torch.square(v_b_pred - self.v_b))

            total_loss = bc_loss + diff_loss
            return diff_loss, bc_loss, total_loss

    def train(self, epochs):
        self.epochs = epochs
        # Define the model
        model = neural_network_model()
        # Define the loss function
        optimizer = torch.optim.Adam(model.parameters())
        model.train()
        
        total_loss_list = []
        BC_loss_list = []
        diff_loss_list = []
        epochs_list = []

        x_col = x[:r]
        y_col = y[:r]
        x_b = x[r:]
        y_b = y[r:]
    
        for e in range(1, epochs+1):
            
            optimizer.zero_grad()
            u_col_pred, v_col_pred = model(x_col, y_col)
            u_b_pred, v_b_pred = model(x_b, y_b)
            
            diff_loss, bc_loss, total_loss = self.loss(x, y, u_col_pred, v_col_pred, u_b_pred, v_b_pred)
            total_loss.backward()
            optimizer.step()

            if e % 50 == 0:
                total_loss_list.append(total_loss.item())
                BC_loss_list.append(bc_loss.item())
                diff_loss_list.append(diff_loss.item())
                epochs_list.append(e)
                print(f'Epoch {e+0:03}: | Total Loss: {total_loss.item():.10f} | Diff Loss: {diff_loss.item():.10f} | BC Loss: {bc_loss.item():.10f}')
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=epochs_list, y=total_loss_list, mode='lines+markers', name='Total Loss'))
        fig.add_trace(go.Scatter(x=epochs_list, y=BC_loss_list, mode='lines+markers', name='BC Loss'))
        fig.add_trace(go.Scatter(x=epochs_list, y=diff_loss_list, mode='lines+markers', name='Diff Loss'))
        fig.update_layout(title='Loss vs Epochs', xaxis_title='Epochs', yaxis_title='Loss')
        fig.show()

        return model
        

In [6]:
pinn_model = PINN_Model(x, y, u_b, v_b, r)
model = pinn_model.train(epochs=10000) # function defined in custom class for training the model

Epoch 050: | Total Loss: 0.3596083522 | Diff Loss: 0.3304856420 | BC Loss: 0.0291227065
Epoch 100: | Total Loss: 0.1211681068 | Diff Loss: 0.1149918586 | BC Loss: 0.0061762468
Epoch 150: | Total Loss: 0.0364493690 | Diff Loss: 0.0340917744 | BC Loss: 0.0023575949
Epoch 200: | Total Loss: 0.0209325105 | Diff Loss: 0.0190562680 | BC Loss: 0.0018762418
Epoch 250: | Total Loss: 0.0130810458 | Diff Loss: 0.0117605850 | BC Loss: 0.0013204606
Epoch 300: | Total Loss: 0.0072104163 | Diff Loss: 0.0065941322 | BC Loss: 0.0006162842
Epoch 350: | Total Loss: 0.0042330697 | Diff Loss: 0.0035058751 | BC Loss: 0.0007271948
Epoch 400: | Total Loss: 0.0029789750 | Diff Loss: 0.0020952686 | BC Loss: 0.0008837065
Epoch 450: | Total Loss: 0.0023076856 | Diff Loss: 0.0014013161 | BC Loss: 0.0009063694
Epoch 500: | Total Loss: 0.0019338576 | Diff Loss: 0.0010589632 | BC Loss: 0.0008748943
Epoch 550: | Total Loss: 0.0016515214 | Diff Loss: 0.0008387194 | BC Loss: 0.0008128020
Epoch 600: | Total Loss: 0.00148

In [7]:
# Saving the model
torch.save(model, 'lab_11_model.pt')

In [8]:
''' Testing PINN '''

# load the model
model = torch.load('lab_11_model.pt')

epsilon = 10e-6
# Generate collocation points and use trained model for testing
x_temp = np.linspace(0+epsilon, 1-epsilon, 20)
y_temp= np.linspace(0+epsilon, 1-epsilon, 20)
x_col_test, y_col_test = np.meshgrid(x_temp, y_temp)
x_col_test = x_col_test.flatten().reshape(-1, 1)
y_col_test = y_col_test.flatten().reshape(-1, 1)
x_col_test = torch.tensor(x_col_test, dtype=torch.float32)
y_col_test = torch.tensor(y_col_test, dtype=torch.float32)

# Generate boundary points and use trained model for testing
x_temp = np.linspace(0, 1, 20)
y_zeros = np.zeros(20)
y_ones = np.ones(20)
y_temp= np.linspace(0, 1, 20)
x_zeros = np.zeros(20)
x_ones = np.ones(20)
x_b_test = np.concatenate((x_temp, x_temp, x_zeros, x_ones)).reshape(-1, 1)
y_b_test = np.concatenate((y_zeros, y_ones, y_temp, y_temp)).reshape(-1, 1)
x_b_test = torch.tensor(x_b_test, dtype=torch.float32)
y_b_test = torch.tensor(y_b_test, dtype=torch.float32)
r = len(x_col_test)

# Use trained model for testing
model.eval()
u_col_pred, v_col_pred = model(x_col_test, y_col_test)
u_b_pred, v_b_pred = model(x_b_test, y_b_test)

# diff_loss, bc_loss, total_loss = PINN_Model.loss( _ ,x, y, u_col_pred, v_col_pred, u_b_pred, v_b_pred)
# print(f'Total Loss: {total_loss.item():.8f} | Diff Loss: {diff_loss.item():.8f} | BC Loss: {bc_loss.item():.8f}')

In [9]:
''' Plotting '''
# Plot the results in terms of contour plots
x_col_test = x_col_test.detach().numpy()
x_col_test = x_col_test.reshape(len(x_col_test))
y_col_test = y_col_test.detach().numpy()
y_col_test = y_col_test.reshape(len(y_col_test))
u_col_pred = u_col_pred.detach().numpy()
u_col_pred = u_col_pred.reshape(len(u_col_pred))
v_col_pred = v_col_pred.detach().numpy()
v_col_pred = v_col_pred.reshape(len(v_col_pred))
fig = go.Figure(data =go.Contour(x=x_col_test, y=y_col_test, z=u_col_pred))
fig.update_layout(width=750,height=700)
fig.show()
fig = go.Figure(data =go.Contour(x=x_col_test, y=y_col_test, z=v_col_pred))
fig.update_layout(width=750,height=700)
fig.show()